In [ ]:
import os
import cv2
import numpy as np
import csv

In [14]:
def process_image_batch(input_dir, crop_coords, output_dir='masks', csv_filename='pixel_counts.csv'):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Get crop coordinates based on one sample image
    original_image_path = os.path.join(input_dir, os.listdir(input_dir)[0])
    
    
    x_start, y_start, x_end, y_end = crop_coords

    # Initialize the CSV file
    with open(csv_filename, mode='w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['File Name', 'Pixel Count'])

        # Iterate over each file in the input directory
        for filename in os.listdir(input_dir):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                image_path = os.path.join(input_dir, filename)
                status = process_single_image(image_path, output_dir, writer, (x_start, y_start, x_end, y_end))
                print(f"Processed {filename}: {status}")

def process_single_image(image_path, output_dir, csv_writer, crop_coords):
    x_start, y_start, x_end, y_end = crop_coords
    
    # Read and crop the image
    image = cv2.imread(image_path)
    if image is None:
        return "Failed to load image"
    
    # Apply cropping
    cropped_image = image[y_start:y_end, x_start:x_end]
    
    # Convert BGR to HSV
    hsv_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2HSV)

    # Define the HSV ranges for green, yellow, and tan colors
    lower_green, upper_green = (35, 40, 40), (85, 255, 255)
    lower_yellow, upper_yellow = (20, 40, 40), (35, 255, 255)
    
    tan_bgr = np.uint8([[[103, 137, 157]]])
    tan_hsv = cv2.cvtColor(tan_bgr, cv2.COLOR_BGR2HSV)
    
    # Use the tan HSV value to define a range around it
    hue = int(tan_hsv[0][0][0])
    sat = int(tan_hsv[0][0][1])
    val = int(tan_hsv[0][0][2])

    # Define the lower and upper ranges for the tan color
    lower_tan = (hue - 10, max(0, sat - 40), int(0.35 * 255))  # Ensure values don't go below 0
    upper_tan = (hue + 10, min(255, sat + 40), int(0.72 * 255))  # Ensure values don't exceed 255

    # Create masks
    green_mask = cv2.inRange(hsv_image, lower_green, upper_green)
    yellow_mask = cv2.inRange(hsv_image, lower_yellow, upper_yellow)
    tan_mask = cv2.inRange(hsv_image, lower_tan, upper_tan)
    
    mask_combined = cv2.bitwise_or(cv2.bitwise_or(green_mask, yellow_mask), tan_mask)

    # Clean the mask using morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask_clean = cv2.morphologyEx(mask_combined, cv2.MORPH_OPEN, kernel)

    # Filter contours by size
    contours, _ = cv2.findContours(mask_clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filtered_mask = np.zeros_like(mask_clean)
    min_size = 3500
    for cnt in contours:
        if cv2.contourArea(cnt) >= min_size:
            temp_mask = np.zeros_like(mask_clean)
            cv2.drawContours(temp_mask, [cnt], 0, 255, -1)
            filtered_mask = cv2.bitwise_or(filtered_mask, cv2.bitwise_and(temp_mask, mask_clean))

    # Save the binary filtered mask image
    mask_filename = os.path.join(output_dir, os.path.splitext(os.path.basename(image_path))[0] + '_mask.png')
    cv2.imwrite(mask_filename, filtered_mask)

    # Count annotated pixels and write to CSV
    filename_without_extension = os.path.splitext(os.path.basename(image_path))[0]
    annotated_pixel_count = np.count_nonzero(filtered_mask)
    csv_writer.writerow([filename_without_extension, annotated_pixel_count])

    return "Success"


In [29]:
# Crop Cood (x_start, y_start, x_end, y_end)
crop_coords = (136, 850, 2887, 3700)
process_image_batch(input_dir='/Users/jshen/Library/CloudStorage/OneDrive-UniversityofManitoba/General - Whyard - Machine Learning/PHOTOS for AI Machine Learning/April 2024 assay/May 03 2024 - Day 14', crop_coords=crop_coords, output_dir='masks_day_14', csv_filename='pixel_counts_day_14.csv')

Processed 20240503_094657.jpg: Success
Processed 20240503_094454.jpg: Success
Processed 20240503_094245.jpg: Success
Processed 20240503_095200.jpg: Success
Processed 20240503_094608.jpg: Success
Processed 20240503_094437.jpg: Success
Processed 20240503_094418.jpg: Success
Processed 20240503_094801.jpg: Success
Processed 20240503_094143.jpg: Success
Processed 20240503_094359.jpg: Success
Processed 20240503_094628.jpg: Success
Processed 20240503_095246.jpg: Success
Processed 20240503_132614.jpg: Success
Processed 20240503_094159.jpg: Success
Processed 20240503_095045.jpg: Success
Processed 20240503_132228.jpg: Success
Processed 20240503_133120.jpg: Success
Processed 20240503_132404.jpg: Success
Processed 20240503_094918.jpg: Success
Processed 20240503_133225.jpg: Success
Processed 20240503_133025.jpg: Success
Processed 20240503_095026.jpg: Success
Processed 20240503_132514.jpg: Success
Processed 20240503_132854.jpg: Success
Processed 20240503_094301.jpg: Success
Processed 20240503_094317

In [8]:
# Usage
process_image_batch('/Users/jshen/Library/CloudStorage/OneDrive-UniversityofManitoba/General - Whyard - Machine Learning/PHOTOS for AI Machine Learning/April 2024 assay/April 19 2024 - Day 0')

Processed 20240419_131423.jpg: Success with :/Users/jshen/Library/CloudStorage/OneDrive-UniversityofManitoba/General - Whyard - Machine Learning/PHOTOS for AI Machine Learning/April 2024 assay/April 19 2024 - Day 0/20240419_131423.jpg
Processed 20240419_104301.jpg: Success with :/Users/jshen/Library/CloudStorage/OneDrive-UniversityofManitoba/General - Whyard - Machine Learning/PHOTOS for AI Machine Learning/April 2024 assay/April 19 2024 - Day 0/20240419_104301.jpg
Processed 20240419_104711.jpg: Success with :/Users/jshen/Library/CloudStorage/OneDrive-UniversityofManitoba/General - Whyard - Machine Learning/PHOTOS for AI Machine Learning/April 2024 assay/April 19 2024 - Day 0/20240419_104711.jpg
Processed 20240419_104658.jpg: Success with :/Users/jshen/Library/CloudStorage/OneDrive-UniversityofManitoba/General - Whyard - Machine Learning/PHOTOS for AI Machine Learning/April 2024 assay/April 19 2024 - Day 0/20240419_104658.jpg
Processed 20240419_140218.jpg: Success with :/Users/jshen/Li

KeyboardInterrupt: 